#### Define settings

In [1]:
sample_n = 50
acc_rate = 0.2
# the number of chosen samples
top_n = int(sample_n * acc_rate)

CPU_n = 4


In [2]:
import time
class clock:
    start_t = 0
    end_t = 0
    @staticmethod
    def start():
        clock.start_t = time.time()
    @staticmethod
    def end():
        clock.end_t = time.time()
        print('Elapsed time: ',clock.end_t - clock.start_t)

##### Define free parameters

In [2]:
free_params = {
    "B_MSC_Pr":[0.001,0.05],
#     "B_MSC_Mo":[0.001,0.005]
}
free_params_keys = list(free_params.keys())
free_params_bounds = list(free_params.values())


##### Sample from free parameters

In [4]:
from diversipy import lhd_matrix
from diversipy import transform_spread_out
import numpy as np
# python version > 3.6
non_scalled_samples = transform_spread_out(lhd_matrix(sample_n, len(free_params))).transpose()
scaled_samples = []
ii = 0
for bounds in free_params_bounds:
    low = bounds[0]
    high = bounds[1]
    
    pre_samples_param = non_scalled_samples[ii]
    
    samples_param = list(map(lambda x:x*(high-low)+low ,pre_samples_param))
    scaled_samples.append(samples_param)
    ii+=1
priors = {key:value for key,value in zip(free_params_keys,scaled_samples)}
samples = np.array(scaled_samples).transpose()


In [5]:
##### create parameter sets

In [6]:
param_sets = []
for sample in samples:
    param_set = {}
    for i in range(len(sample)):
        sample_p = sample[i]
        key = free_params_keys[i]
        param_set.update({key:sample_p})
    param_sets.append(param_set)

[{'B_MSC_Pr': 0.032},
 {'B_MSC_Pr': 0.002},
 {'B_MSC_Pr': 0.027000000000000003},
 {'B_MSC_Pr': 0.007},
 {'B_MSC_Pr': 0.022000000000000002},
 {'B_MSC_Pr': 0.044000000000000004},
 {'B_MSC_Pr': 0.010000000000000002},
 {'B_MSC_Pr': 0.011},
 {'B_MSC_Pr': 0.023000000000000003},
 {'B_MSC_Pr': 0.019000000000000003},
 {'B_MSC_Pr': 0.006},
 {'B_MSC_Pr': 0.004},
 {'B_MSC_Pr': 0.017},
 {'B_MSC_Pr': 0.031000000000000003},
 {'B_MSC_Pr': 0.049},
 {'B_MSC_Pr': 0.045000000000000005},
 {'B_MSC_Pr': 0.048},
 {'B_MSC_Pr': 0.037000000000000005},
 {'B_MSC_Pr': 0.029},
 {'B_MSC_Pr': 0.014000000000000002},
 {'B_MSC_Pr': 0.042},
 {'B_MSC_Pr': 0.001},
 {'B_MSC_Pr': 0.04},
 {'B_MSC_Pr': 0.05},
 {'B_MSC_Pr': 0.033},
 {'B_MSC_Pr': 0.012},
 {'B_MSC_Pr': 0.021},
 {'B_MSC_Pr': 0.015},
 {'B_MSC_Pr': 0.003},
 {'B_MSC_Pr': 0.030000000000000002},
 {'B_MSC_Pr': 0.028},
 {'B_MSC_Pr': 0.013000000000000001},
 {'B_MSC_Pr': 0.024000000000000004},
 {'B_MSC_Pr': 0.039},
 {'B_MSC_Pr': 0.005},
 {'B_MSC_Pr': 0.034},
 {'B_MSC_Pr': 0

##### Import the scheme: 
This contains configs, the returns, and the emperical data associated with each run

In [7]:
import json
SCHEMES_PATH = "schemes.json"
with open(SCHEMES_PATH) as schemes_file:
    schemes = json.load(schemes_file)["schemes"]


**Extract expectations from schemes**

In [8]:
expectations = [scheme["expectations"] for scheme in schemes]

[{'liveCellCount': [100, 160, 270], 'viability': [0.95]}]

**Define distance function** 

In [9]:

def distance_function(realizations,expectations):
    distances = []
    for i in range(len(realizations)):
        raw_sim_data = realizations[i]
        sim_data = {}
        sim_liveCellCount = np.array(raw_sim_data["agents_count"]["MSC"])
        sim_totalCellCount = np.array(raw_sim_data["agents_count"]["MSC"]) + np.array(raw_sim_data["agents_count"]["Dead"])
        sim_viability = sim_liveCellCount[-1]/sim_totalCellCount[-1] 
        sim_data.update({"liveCellCount":sim_liveCellCount,
                         "viability": sim_viability})
                
        exp_data = expectations[i]

        error_liveCellCount = (sim_data["liveCellCount"] - np.array(exp_data["liveCellCount"]))/np.array(exp_data["liveCellCount"])
        error_viability = (sim_data["viability"] - np.array(exp_data["viability"]))/np.array(exp_data["viability"])

        all_errors = np.concatenate((error_liveCellCount,error_viability),axis=0)
        abs_all_errors = np.abs(all_errors)

        distances.append(np.mean(abs_all_errors)) 
    
    distance = np.mean(distances)
    

    return distance


**Import the model**

In [10]:
import os,sys
sys.path.insert(1, '/Users/matin/Downloads/testProjs/CA')
from model import Model

**Run the model for each param set**

In [11]:
import copy

def run_model(args):
    start_n = args[0]
    end_n = args[1]
    distances = np.array([])
    for i in range(start_n,end_n):
        param_set = param_sets[i]
#         print("iteration ",i)
        schemes_copy = copy.deepcopy(schemes)
        sim_results_list = Model(param_set).run(schemes_copy)
        distance = distance_function(sim_results_list,expectations)
        distances = np.append(distances,distance)
    return distances

Run the model for each param set in parallel

In [12]:
from multiprocessing import Pool

share = int(sample_n/CPU_n)
plus =  sample_n%CPU_n

indices =[np.array([x,x+1])*share for x in range(CPU_n)] 
# The last one gets more share
indices[-1][1]+=plus

pl = Pool(CPU_n)
clock.start()
results = pl.map(run_model,indices)
clock.end()


Elapsed time:  44.684529066085815


Concatenate the results coming from differen CPUs

In [13]:
distances = [] 
for result in results:
    distances = np.concatenate([distances,result],axis=0)

## Post processing:
- Extract the indices of min n values of distances
- Extract the best fits
- Extract associated samples
- Create posteriors
- Save posteriors

In [38]:
import json
top_ind = np.argpartition(distances, top_n)[:top_n]
top_fit_distances = distances[top_ind]
top_fit_samples = samples[top_ind]
posteriors = {key:list(value) for key,value in zip(free_params_keys,top_fit_samples.transpose())}

with open('outputs/posterior.json', 'w') as file:
     file.write(json.dumps({'posteriors': posteriors}))


{'B_MSC_Pr': [0.015, 0.019000000000000003, 0.018000000000000002, 0.014000000000000002, 0.012, 0.016, 0.013000000000000001, 0.02, 0.023000000000000003, 0.017]}


### scale the posterior

In [3]:
import json
with open('outputs/posterior.json') as file:
    posteriors = json.load(file)["posteriors"]

scalled_posteriors = {}
for key,values in posteriors.items():
    min_v = free_params[key][0]
    max_v = free_params[key][1]
    scalled = list(map(lambda x: (x-min_v)/(max_v-min_v),values))
    scalled_posteriors.update({key:scalled})

### Box Plot: median, quartiles, and raw data

In [6]:
import plotly.graph_objects as go
import plotly.offline
# init_notebook_mode(connected=True)
colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)', 'rgba(44, 160, 101, 0.5)',
          'rgba(255, 65, 54, 0.5)', 'rgba(207, 114, 255, 0.5)', 'rgba(127, 96, 0, 0.5)']
traces = []
ii = 0
for key,value in scalled_posteriors.items():
    traces.append(go.Box(
        y=value,
        name=key,
        boxpoints='all',
        jitter=0,
        fillcolor=colors[ii],
        marker_size=5,
        whiskerwidth=0.2,
        line_width=2)
                 )
    ii += 1
layout = go.Layout(yaxis=dict(
#                             autorange=True,
#                             showgrid=False,
                            dtick=0.2,
                            zeroline = False,range= [-0.1,1.1]
                            ),
                    margin=dict(
                            l=40,
                            r=30,
                            b=80,
                            t=100
                        ),
                    showlegend=False,
                    paper_bgcolor='rgb(243, 243, 243)',
                    plot_bgcolor='rgb(243, 243, 243)',
                   )
fig = { "data": traces,"layout":layout }
plotly.io.write_image(fig = { "data": traces,"layout":layout }, file="outputs/box_plot.svg",format="svg",scale=None, width=None, height=None)
# plotly.offline.plot(fig = { "data": traces,"layout":layout }, auto_open = True)
